In [10]:
import torch
from torchview import draw_graph
from generator.parser import Parser
import generator.bricks as bricks
import sys
import time
import torch.nn.functional as F

ModuleNotFoundError: No module named 'generator'

In [13]:
device = "cuda"
parser = Parser()
s1 = "output={{@4->relu+@8->relu}^2}%2->@16->softmax->linear(5);"
s2 = "output={{@16->relu+@16->sigmoid}^4}%8->@16;"
s3 = "output={{@64->relu}^64}%64;"
s4 = "output = linear(5) -> softmax;"
s5 = "output = {@5 + @10 + @20} -> softmax;"
start_time = time.time()
modules = parser.from_str(s5)
# modules = parser.from_json('nntest.json')
model = modules['output'].to(device)
end_time = time.time()


В модуле не определен входной блок


In [20]:
# Тестируем работу сети
output2 = modules['output'].to(device)(torch.randn(1, 10).to(device))
print(output2)
# Получим один из элементов модели
print(modules['output'].left)
print(modules['output'].left.left.right)
modules

tensor([[0.0299, 0.0163, 0.0452, 0.0487, 0.0284, 0.0277, 0.0178, 0.0710, 0.0233,
         0.0087, 0.0425, 0.0222, 0.0284, 0.0102, 0.0270, 0.0211, 0.0293, 0.0160,
         0.0456, 0.0446, 0.0213, 0.0381, 0.0311, 0.0380, 0.0103, 0.0309, 0.0221,
         0.0423, 0.0282, 0.0231, 0.0173, 0.0215, 0.0222, 0.0207, 0.0286]],
       device='cuda:0', grad_fn=<SoftmaxBackward0>)
cat
linear(10)


{'output': MulModule(
   (left): cat
   (right): softmax
 )}

In [5]:
params_count = sum(p.numel() for p in modules['output'].parameters())
print(f"Параметров: {params_count}, время создания: ", end_time - start_time)
output2.shape

Параметров: 385, время создания:  0.020548343658447266


torch.Size([1, 35])

In [6]:
model.expr_str(expand=False)

'{{{linear(5)+linear(10)}+linear(20)}->softmax}'

In [7]:
model_graph = draw_graph(
    model, 
    input_size=(1, 10),
    graph_name='test',
    graph_dir='LR',
    depth=8,
    hide_inner_tensors=True,
    hide_module_functions=True,
    save_graph=True,
    expand_nested=True,
    show_shapes=True,
    filename='test',
    directory='./pic/'
)

# model_graph.resize_graph(scale=3)
# model_graph.visual_graph.view('./pic/test',)

In [8]:
x = torch.randn(1,10).to(device)
chunk = modules['output'].left.right
print(chunk(x))
model_graph = draw_graph(
    chunk, 
    input_size=(1, 10),
    graph_name='left.right',
    graph_dir='LR',
    depth=8,
    hide_inner_tensors=True,
    hide_module_functions=True,
    save_graph=True,
    expand_nested=True,
    show_shapes=True,
    filename='left.right',
    directory='./pic/'
)
left, right = chunk.decompose()
new_chunk = bricks.Multiplicator(left, right).to("cuda")
print(new_chunk(x))
model_graph = draw_graph(
    new_chunk, 
    input_size=(1, 10),
    graph_name='decompose',
    graph_dir='LR',
    depth=8,
    hide_inner_tensors=True,
    hide_module_functions=True,
    save_graph=True,
    expand_nested=True,
    show_shapes=True,
    filename='decompose',
    directory='./pic/'
)

tensor([[ 0.9356,  1.0309, -0.1638, -0.4745, -1.0700, -0.6119, -0.1854,  0.6896,
         -1.3516,  0.3454,  0.7736,  0.4970, -0.8980,  0.6620,  0.7820,  0.6722,
          0.1421, -0.2478, -0.7681, -1.2987]], device='cuda:0',
       grad_fn=<AddmmBackward0>)
tensor([[ 0.9356,  1.0309, -0.1638, -0.4745, -1.0700, -0.6119, -0.1854,  0.6896,
         -1.3516,  0.3454,  0.7736,  0.4970, -0.8980,  0.6620,  0.7820,  0.6722,
          0.1421, -0.2478, -0.7681, -1.2987]], device='cuda:0',
       grad_fn=<AddmmBackward0>)
